In [ ]:
import torch
from torch import Tensor
print(torch.__version__)

2.0.0+cu118


In [ ]:
# Install required packages.
import os
os.environ['TORCH'] = torch.__version__

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/CSE6240/notebooks

/content/drive/.shortcut-targets-by-id/1y1Lvxa-gIYQlLQs7Dzoz9XVXO1BPTpP5/CSE6240/notebooks


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch_geometric import nn
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T

In [ ]:
# Load data
test_edges = pd.read_parquet('../data/graphs/test_edges.parquet')
test_labels = pd.read_parquet('../data/graphs/test_labels_neg.parquet')

val_edges = pd.read_parquet('../data/graphs/val_edges.parquet')
val_labels = pd.read_parquet('../data/graphs/val_labels_neg.parquet')

train_edges = pd.read_parquet('../data/graphs/train_edges.parquet')
train_labels = pd.read_parquet('../data/graphs/train_labels_neg.parquet')

In [ ]:
prod_features = np.load('../data/feature_emb/products.npy')
user_features = np.load('../data/feature_agg/train_user_features_norm.npy')

In [ ]:
enc_user_id = train_edges[["user_id"]].drop_duplicates().sort_values("user_id").reset_index(drop=True).reset_index().rename(columns={"index": "enc_user_id"})
print(enc_user_id.shape)
enc_user_id.head()

(101696, 2)


,enc_user_id,user_id
0,0,1
1,1,2
2,2,3
3,3,7
4,4,13


In [ ]:
enc_user_id_dict = dict(zip(enc_user_id.user_id, enc_user_id.enc_user_id))
enc_user_id_dict[7]

3

In [ ]:
# encode user_id
train_edges["enc_user_id"] = train_edges["user_id"].map(enc_user_id_dict)
val_edges["enc_user_id"] = val_edges["user_id"].map(enc_user_id_dict)
test_edges["enc_user_id"] = test_edges["user_id"].map(enc_user_id_dict)
train_labels["enc_user_id"] = train_labels["user_id"].map(enc_user_id_dict)
val_labels["enc_user_id"] = val_labels["user_id"].map(enc_user_id_dict)
test_labels["enc_user_id"] = test_labels["user_id"].map(enc_user_id_dict)


In [ ]:
train_edges

,user_id,product_id,weight,enc_user_id
0,1,196,0.700000,0
1,1,10258,0.600000,0
2,1,10326,0.100000,0
3,1,12427,0.700000,0
4,1,13032,0.200000,0
...,...,...,...,...
8675716,206209,41665,0.076923,101695
8675717,206209,43961,0.153846,101695
8675718,206209,44325,0.076923,101695
8675719,206209,48697,0.076923,101695


In [ ]:
train_edge_index = torch.tensor(train_edges[["enc_user_id", "product_id"]].values, 
                                dtype=torch.long).t().contiguous()
train_data = HeteroData()

# Save node indices:
train_data["user"].node_id = torch.arange(user_features.shape[0])
train_data["prod"].node_id = torch.arange(prod_features.shape[0])


# Add the node features and edge indices:
train_data["user"].x = torch.tensor(user_features, dtype=torch.float32)
train_data["prod"].x = torch.tensor(prod_features, dtype=torch.float32)
train_data["user", "buy", "prod"].edge_index = train_edge_index

train_data = T.ToUndirected()(train_data)
print(train_data)

In [ ]:
train_data["user"].x

tensor([[-0.8741,  1.4756, -2.2054,  ..., -0.8210, -1.0658, -1.2879],
        [-0.6572,  1.4756, -0.8575,  ...,  0.6467, -0.7654,  0.6558],
        [-0.7657, -0.4206, -3.5534,  ..., -0.3859, -0.9156, -0.9345],
        ...,
        [-0.5487, -0.4206,  0.4904,  ...,  0.6677, -0.6152,  0.6599],
        [ 1.2412, -0.4206,  0.4904,  ...,  0.6372,  1.8628,  0.4286],
        [-0.7114, -0.4206, -0.8575,  ..., -0.1540, -0.8405,  0.2472]])

HeteroData(
  user={
    node_id=[101696],
    x=[101696, 22]
  },
  prod={
    node_id=[49689],
    x=[49689, 768]
  },
  (user, buy, prod)={ edge_index=[2, 8675721] },
  (prod, rev_buy, user)={ edge_index=[2, 8675721] }
)


In [ ]:
train_data["user", "buy", "prod"].edge_label_index = torch.tensor(
    train_labels[["enc_user_id", "product_id"]].values, dtype=torch.long).t().contiguous()
train_data["user", "buy", "prod"].edge_label = torch.tensor(
    train_labels["label"].values, dtype=torch.float32).t().contiguous()
train_data

HeteroData(
  user={
    node_id=[101696],
    x=[101696, 22]
  },
  prod={
    node_id=[49689],
    x=[49689, 768]
  },
  (user, buy, prod)={
    edge_index=[2, 8675721],
    edge_label_index=[2, 661052],
    edge_label=[661052]
  },
  (prod, rev_buy, user)={ edge_index=[2, 8675721] }
)

In [ ]:
# In the first hop, we sample at most 20 neighbors.
# In the second hop, we sample at most 10 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data["user", "buy", "prod"].edge_label_index
edge_label = train_data["user", "buy", "prod"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), edge_label_index),
    edge_label=edge_label,
    batch_size=512,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

print(sampled_data["user", "buy", "prod"].edge_label_index.size(1))
print(sampled_data["user", "buy", "prod"].edge_label.min())
print(sampled_data["user", "buy", "prod"].edge_label.max())

Sampled mini-batch:
HeteroData(
  user={
    node_id=[79331],
    x=[79331, 22],
    n_id=[79331]
  },
  prod={
    node_id=[36540],
    x=[36540, 768],
    n_id=[36540]
  },
  (user, buy, prod)={
    edge_index=[2, 307401],
    edge_label_index=[2, 512],
    edge_label=[512],
    e_id=[307401],
    input_id=[512]
  },
  (prod, rev_buy, user)={
    edge_index=[2, 689997],
    e_id=[689997]
  }
)
512
tensor(0.)
tensor(1.)


In [ ]:
sampled_data["user", "buy", "prod"].edge_label_index

tensor([[ 14, 301, 260,  ..., 492,  44, 472],
        [ 44, 226, 192,  ..., 156, 489,  27]])

In [ ]:
sampled_data["user"].x

tensor([[-0.6572, -0.4206,  0.4904,  ...,  0.9682, -0.7654,  1.0046],
        [-0.3317, -0.4206,  0.4904,  ..., -0.4286, -0.3148, -0.6334],
        [ 0.4818, -0.4206,  0.4904,  ...,  0.6396,  0.8115,  1.0200],
        ...,
        [-0.7114, -0.4206,  0.4904,  ..., -0.2070, -0.8405, -0.4902],
        [-0.2233, -0.4206,  0.4904,  ..., -0.6827, -0.1646, -0.1278],
        [-0.4402, -0.4206,  0.4904,  ..., -0.2719, -0.4650, -0.3227]])

In [ ]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def __init__(self, hidden_channels, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.lin_cls_1 = torch.nn.Linear(hidden_channels, 512)
        self.lin_cls_2 = torch.nn.Linear(512, 1)
    def forward(self, x_user: Tensor, x_prod: Tensor, edge_label_index: Tensor
                ) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_prod = x_prod[edge_label_index[1]]
        out = edge_feat_user * edge_feat_prod
        out = F.relu(self.lin_cls_1(out))
        out = self.lin_cls_2(out)

        # Apply dot-product to get a prediction per supervision edge:
        return out.squeeze()


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        self.prod_lin = torch.nn.Linear(prod_features.shape[1], hidden_channels)
        self.user_lin = torch.nn.Linear(user_features.shape[1], hidden_channels)
        self.user_emb = torch.nn.Embedding(train_data["user"].num_nodes, hidden_channels)
        self.prod_emb = torch.nn.Embedding(train_data["prod"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=train_data.metadata())

        self.classifier = Classifier(hidden_channels)

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "user": self.user_lin(data["user"].x) + self.user_emb(data["user"].node_id),
          "prod": self.prod_lin(data["prod"].x) + self.prod_emb(data["prod"].node_id),
        } 

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["prod"],
            data["user", "buy", "prod"].edge_label_index,
        )

        return pred

        
model = Model(hidden_channels=128)

print(model)

Model(
  (prod_lin): Linear(in_features=768, out_features=128, bias=True)
  (user_lin): Linear(in_features=22, out_features=128, bias=True)
  (user_emb): Embedding(101696, 128)
  (prod_emb): Embedding(49689, 128)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__buy__prod): SAGEConv(128, 128, aggr=mean)
      (prod__rev_buy__user): SAGEConv(128, 128, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__buy__prod): SAGEConv(128, 128, aggr=mean)
      (prod__rev_buy__user): SAGEConv(128, 128, aggr=mean)
    )
    (conv3): ModuleDict(
      (user__buy__prod): SAGEConv(128, 128, aggr=mean)
      (prod__rev_buy__user): SAGEConv(128, 128, aggr=mean)
    )
  )
  (classifier): Classifier(
    (lin_cls_1): Linear(in_features=128, out_features=512, bias=True)
    (lin_cls_2): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [ ]:
import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        sampled_data.to(device)
        pred = model(sampled_data)

        ground_truth = sampled_data["user", "buy", "prod"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


100%|██████████| 1292/1292 [06:56<00:00,  3.10it/s]


Epoch: 001, Loss: 0.3962


100%|██████████| 1292/1292 [06:57<00:00,  3.10it/s]


Epoch: 002, Loss: 0.3245


...

Epoch: 007, Loss: 0.2351


100%|██████████| 1292/1292 [07:01<00:00,  3.07it/s]


Epoch: 008, Loss: 0.2179


100%|██████████| 1292/1292 [07:01<00:00,  3.06it/s]

Epoch: 009, Loss: 0.2011


In [ ]:
with open("../saved_model/graphsage_128_v2.pkl", "wb") as f:
    torch.save(model, f=f)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-79-50a464592d25>", line 1, in <cell line: 1>
    with open("../saved_model/graphsage_128_v2_1.pkl", "wb") as f:
OSError: [Errno 107] Transport endpoint is not connected: '../saved_model/graphsage_128_v2_1.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_o

In [ ]:
def prep_dataset(df_edges, df_labels):
    edge_index = torch.tensor(df_edges[["enc_user_id", "product_id"]].values, 
                                dtype=torch.long).t().contiguous()
    data = HeteroData()

    # Save node indices:
    data["user"].node_id = torch.arange(user_features.shape[0])
    data["prod"].node_id = torch.arange(prod_features.shape[0])


    # Add the node features and edge indices:
    data["user"].x = torch.tensor(user_features, dtype=torch.float32)
    data["prod"].x = torch.tensor(prod_features, dtype=torch.float32)
    data["user", "buy", "prod"].edge_index = edge_index

    data = T.ToUndirected()(data)
    data["user", "buy", "prod"].edge_label_index = torch.tensor(
        df_labels[["enc_user_id", "product_id"]].values, dtype=torch.long).t().contiguous()
    data["user", "buy", "prod"].edge_label = torch.tensor(
        df_labels["label"].values, dtype=torch.float32).t().contiguous()
    return data

In [ ]:
val_data = prep_dataset(val_edges, val_labels)
val_data

HeteroData(
  user={
    node_id=[101696],
    x=[101696, 22]
  },
  prod={
    node_id=[49689],
    x=[49689, 768]
  },
  (user, buy, prod)={
    edge_index=[2, 9006247],
    edge_label_index=[2, 661678],
    edge_label=[661678]
  },
  (prod, rev_buy, user)={ edge_index=[2, 9006247] }
)

In [ ]:
# Define the validation seed edges:
edge_label_index = val_data["user", "buy", "prod"].edge_label_index
edge_label = val_data["user", "buy", "prod"].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10, 20],
    edge_label_index=(("user", "buy", "prod"), edge_label_index),
    edge_label=edge_label,
    batch_size=2048,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

print(sampled_data["user", "buy", "prod"].edge_label_index.size(1))
print(sampled_data["user", "buy", "prod"].edge_label.min() )
print(sampled_data["user", "buy", "prod"].edge_label.max() )

Sampled mini-batch:
HeteroData(
  user={
    node_id=[87217],
    x=[87217, 22],
    n_id=[87217]
  },
  prod={
    node_id=[34379],
    x=[34379, 768],
    n_id=[34379]
  },
  (user, buy, prod)={
    edge_index=[2, 496163],
    edge_label_index=[2, 2048],
    edge_label=[2048],
    e_id=[496163],
    input_id=[2048]
  },
  (prod, rev_buy, user)={
    edge_index=[2, 516728],
    e_id=[516728]
  }
)
2048
tensor(0.)
tensor(1.)


In [79]:
from sklearn.metrics import roc_auc_score

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        sampled_data.to(device)
        preds.append(model(sampled_data))
        ground_truths.append(sampled_data["user", "buy", "prod"].edge_label)

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")

100%|██████████| 324/324 [01:29<00:00,  3.63it/s]



Validation AUC: 0.9202


In [80]:

from sklearn.metrics import roc_auc_score, classification_report

In [81]:
pred_proba = torch.sigmoid(torch.tensor(pred)).cpu().numpy()
pred_binary = (pred_proba > 0.5).astype(int)
print(classification_report(ground_truth, pred_binary))

              precision    recall  f1-score   support

         0.0       0.87      0.81      0.84    330839
         1.0       0.82      0.88      0.85    330839

    accuracy                           0.85    661678
   macro avg       0.85      0.85      0.85    661678
weighted avg       0.85      0.85      0.85    661678

